逼近理想解排序法

In [ ]:
import numpy as np

#从用户输入中接受参评数目和指标数目，并将输入的字符串转换为数值
n = int(input('请输入参评数目：'))
m = int(input('请输入指标数目：'))
# print(n,m)
#接受用户输入的类型矩阵，该矩阵指示了每个指标的类型
print('请输入类型矩阵：1：极大型，2：极小型，3：中间型，4：区间型')
kind = input().split(' ')  #将输入的字符串按空格分割，形成列表

#接收用户输入的矩阵并转化为numpy数组
#A = [9,10,175,120;8,7,164,80;6,3,157,90]
print('请输入矩阵：')
A = np.zeros(shape=(n,m))  #初始化一个n行m列的全零矩阵A
for i in range(n):
    A[i] = input().split(" ")  #接收每行输入的数据
    A[i] = list(map(float,A[i]))  #将接收到的字符串列表转换为浮点数列表
print('输入矩阵为：\n{}'.format(A))

#极小型指标转换为极大型指标
def mintomax(x, maxx):
    x = list(x)  #将输入的指标转化为列表
    ans = [[(maxx-e)] for e in x]  #记算最大值与每个指标的差，并放入新列表中
    return np.array(ans)  #将列表转化为np数组并返回
#中间型指标转换为极大型指标
def midtomax(bestx, x):
    x = list(x)
    h = [abs(e-bestx) for e in x]  #记算每个值与最优值之间的绝对差
    M = max(h)  #找到最大的差值
    if M == 0:
        M = 1  #防止最大值为0
    ans = [[(1-e/M)] for e in h]  #记算每个差值占最大差值的比例，并从1中减去
    return np.array(ans)
#区间型指标转换为极大型指标
def regtomax(lowx, highx, x):
    x = list(x)
    M = max(lowx-min(x), max(x)-highx)  #记算指标值超出区间的最大距离
    if M == 0:
        M =1
    ans = []
    for i in range(len(x)):
        if x[i]<lowx:
            ans.append([(1-(lowx-x[i])/M)])
        elif x[i] > highx:
            ans.append([(1-(x[i]-highx)/M)])
        else:
            ans.append([1])
    return np.array(ans)

#统一指标类型
X = np.zeros(shape=(n,1))
for i in range(m):
    if kind[i] == '1':
        v = np.array(A[:,i])
    elif kind[i] == '2':
        maxA = max(A[:,i])
        v = mintomax(A[:,i], maxA)
    elif kind[i] == '3':
        print('类型三：请输入最优值：')
        bestA = eval(input())
        v = midtomax(bestA, A[:, i])
    elif kind[i] == '4':
        print('类型四：请输入区间[a,b]值a：')
        lowA = eval(input())
        print('类型四：请输入区间[a,b]值b：')
        highA = eval(input())
        v = regtomax(lowA, highA, A[:, i])
    if i == 0:
        X = v.reshape(-1,1)  #如果是第一个指标，直接替换X数组  reshape(-1,*)表示自动识别行数
    else:
        X = np.hstack([X, v.reshape(-1,1)])  #如果不是第一个指标，则将新指标拼接在X数组上
print('统一指标后矩阵为：\n{}'.format(X))

#标准化处理
X = X.astype('float')  #确保矩阵的数据类型为浮点数
for j in range(m):
    X[:,j] = X[:,j]/np.sqrt(sum(X[:,j]**2))
print('标准化矩阵为：\n{}'.format(X))

#最大最小距离的记算
x_max = np.max(X,axis=0)  #每列的最大值
x_min = np.min(X,axis=0)
d_z = np.sqrt(np.sum(np.square((X-np.tile(x_max, reps=(n,1)))), axis=1))
d_f = np.sqrt(np.sum(np.square((X-np.tile(x_min, reps=(n,1)))), axis=1))
print('每个指标的最大值：', x_max)
print('每个指标的最小值：', x_min)
print('d+向量：', d_z)
print('d-向量：', d_f)

#记算每个参评对象的得分排名
s = d_f/(d_z+d_f)
Score = 100*s/sum(s)
for i in range(len(Score)):
    print(f'di {i+1}个标准化后百分制得分为：{Score[i]}')


请输入类型矩阵：1：极大型，2：极小型，3：中间型，4：区间型
请输入矩阵：
输入矩阵为：
[[  9.  10. 175. 120.]
 [  8.   7. 164.  80.]
 [  6.   3. 157.  90.]]
类型三：请输入最优值：
类型四：请输入区间[a,b]值a：
类型四：请输入区间[a,b]值b：
统一指标后矩阵为：
[[9.  0.  0.  0. ]
 [8.  3.  0.9 0.5]
 [6.  7.  0.2 1. ]]
标准化矩阵为：
[[0.66896473 0.         0.         0.        ]
 [0.59463532 0.3939193  0.97618706 0.4472136 ]
 [0.44597649 0.91914503 0.21693046 0.89442719]]
每个指标的最大值： [0.66896473 0.91914503 0.97618706 0.89442719]
每个指标的最小值： [0.44597649 0.         0.         0.        ]
d+向量： [1.61175952 0.69382053 0.79132442]
d-向量： [0.22298824 1.15334862 1.30072534]
di 1个标准化后百分制得分为：8.886366735657832
di 2个标准化后百分制得分为：45.653341055701134
di 3个标准化后百分制得分为：45.46029220864103
